<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Javier Santidrián Salas
- Nombre de alumno 2: Patricio Yáñez Alarcón


### **Link de repositorio de GitHub:** https://github.com/jsantidrian/labs-Santidrian-Yanez

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle("/content/drive/My Drive/U/11 Semestre/Lab Progra/Labs/Lab 7/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Respuesta:**

In [4]:
from datetime import datetime

In [5]:
def custom_features(dataframe_in):
  
  # Agrupamos por cliente (Customer ID)
  clientes = dataframe_in.groupby('Customer ID')

  # Length (L)
  length = (
      (clientes['InvoiceDate'].max() - clientes['InvoiceDate'].min())
      .dt.days
      .reset_index()
  )
  length.rename(columns={'InvoiceDate':'Length'},inplace=True)

  # Recency (R)
  recency = (
      (datetime.now() - clientes['InvoiceDate'].max())
      .dt.days
      .reset_index()
  )
  recency.rename(columns={'InvoiceDate':'Recency'},inplace=True)

  # Monetary (M)
  dataframe_in['Revenue'] = dataframe_in['Price'] * dataframe_in['Quantity']
  monetary = (
      round(clientes['Revenue'].mean(),2).reset_index()
  )
  monetary.rename(columns={'Revenue':'Monetary'},inplace=True)
  dataframe_in.drop('Revenue',axis=1,inplace=True)

  # Frecuency (F)
  frecuency = (
    clientes['InvoiceDate'].count().reset_index()
  )
  frecuency.rename(columns={'InvoiceDate':'Frecuency'},inplace=True)

  # Periodicity (P)
  dataframe_in['Diferencias'] = clientes['InvoiceDate'].diff().dt.days
  periodicity = (
      round(clientes['Diferencias'].std(),0).reset_index()
  )
  periodicity.rename(columns={'Diferencias':'Periodicity'},inplace=True)
  dataframe_in.drop('Diferencias',axis=1,inplace=True)
                                                                                                    
  # LRMFP
  lrmfp = (
      pd.merge(length,recency,on='Customer ID')
      .merge(monetary,on='Customer ID')
      .merge(frecuency,on='Customer ID')
      .merge(periodicity,on='Customer ID')
  )

  # Dropeamos valores NaN para poder operar la data más adelante
  lrmfp.dropna(inplace=True)

  return lrmfp

**Resúesta de Pauta:**

In [6]:
custom_features(df_retail)

,Customer ID,Length,Recency,Monetary,Frecuency,Periodicity
0,12346.0,196,4720,11.30,33,22.0
1,12347.0,37,4558,18.64,71,4.0
2,12348.0,0,4629,11.11,20,0.0
3,12349.0,181,4598,26.19,102,16.0
4,12351.0,0,4566,14.33,21,0.0
...,...,...,...,...,...,...
4309,18283.0,275,4573,2.85,217,12.0
4310,18284.0,0,4622,16.49,28,0.0
4311,18285.0,0,4851,35.58,12,0.0
4312,18286.0,247,4667,19.35,67,30.0


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):

  def __init__(self):
      self.min_vals = None
      self.max_vals = None
  
  def fit(self, X):
      self.min_vals = X.min()
      self.max_vals = X.max()
      return self
  
  def transform(self, X):
      X_transformed = (X - self.min_vals)/(self.max_vals - self.min_vals)
      return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [8]:
# Obtención de caracteristicas LRMFP mediante función custom_features (Paso 1)
caracteristicas = FunctionTransformer(
    func=custom_features
)

# Escalamiento de características obtenidas mediante clase MinMax() (Paso 2)
escalamiento = ColumnTransformer(
    transformers=[
        ('scaler', MinMax(),
         ['Length','Recency','Monetary','Frecuency','Periodicity'])
    ]
)

# Obtención de las 2 componentes más relevantes mediante TSNE (Paso 3)
componentes = TSNE(n_components=2,random_state=0)

# Pipeline
pipeline = Pipeline([
    ('caracteristicas',caracteristicas),
    ('escalamiento', escalamiento),
    ('componentes',componentes)  
])

# Aplicamos el pipeline 
X_tsne = pipeline.fit_transform(df_retail)

# Imprimimos
X_tsne

array([[ -6.0926237,  16.849468 ],
       [ -8.642876 , -39.352886 ],
       [-58.655277 , -26.779863 ],
       ...,
       [ -7.6803646,  46.186344 ],
       [ 10.793542 ,  15.50011  ],
       [  8.90293  , -20.27908  ]], dtype=float32)

In [9]:
# Formamos un dataframe
df_tsne = pd.DataFrame({'Componente 1': X_tsne[:, 0], 'Componente 2': X_tsne[:, 1]})

# Hacemos un scatter plot
fig = px.scatter(df_tsne, x='Componente 1', y='Componente 2')
fig.update_layout(
    title = 'Reducción de dimensionalidad de las características LRMFP mediante t-SNE'
)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [10]:
# Medida de desempeño del clustering (inertia/sse)
inertias = []

# Iteramos sobre las posibles cantidades de clusters k  
for k in range(1,21):

  # K-Means (Paso 3 del pipeline)
  clustering = KMeans(n_clusters=k,random_state=0)

  # Nuevo pipeline (reemplaza t-SNE por K-Means)
  pipeline = Pipeline([
      ('caracteristicas',caracteristicas),
      ('escalamiento', escalamiento),
      ('clustering',clustering)  
  ])
    
  # Ajustamos el clustering a los datos
  pipeline.fit_transform(df_retail)

  # Calculamos el valor de inertia
  inertia = pipeline.named_steps['clustering'].inertia_

  # Agregamos inertia y la cantidad de clusters k a la lista inertias
  inertias.append([k,inertia])

# Imprimimos inertias de acuerdo al número de clusters
inertias

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

[[1, 843.297603434736],
 [2, 342.35739758914883],
 [3, 190.05900547742755],
 [4, 130.39204774667354],
 [5, 110.35816196787013],
 [6, 94.50718719634969],
 [7, 80.14745635766371],
 [8, 70.59313709774923],
 [9, 64.7340066842003],
 [10, 59.32167705082989],
 [11, 54.81543318878257],
 [12, 51.24757522982154],
 [13, 48.27474353494779],
 [14, 45.65230005888543],
 [15, 42.46615485145095],
 [16, 40.25664173195106],
 [17, 38.22067314041627],
 [18, 36.82517643331182],
 [19, 35.22927329038768],
 [20, 33.512897631631844]]

In [11]:
# Agregamos a un dataframe
df_inertias = pd.DataFrame(inertias, columns=["n° clusters", "inertia"])

# Visualizamos el método del codo
fig = px.line(
    df_inertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)
fig.update_traces(mode='lines+markers')
fig.show()

Notamos del gráfico que el valor de $k=3$ es adecuado: en este punto se produce una disminución significativamente menor en el valor de inertia y es el que más se asemeja a un codo (notar que de $k=4$ en adelante los valores de inertia no disminuyen casi nada en comparación a lo que sucede antes de $k=3$).

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [12]:
# K-Means con k=3 optimo segun el metodo del codo (Paso 3 del pipeline)
clustering = KMeans(n_clusters=3,random_state=0)

# Pipeline
pipeline = Pipeline([
    ('caracteristicas',caracteristicas),
    ('escalamiento', escalamiento),
    ('clustering',clustering)  
])
  
# Ajustamos el clustering a los datos
pipeline.fit_transform(df_retail)

# Obtenemos clusters (label 0,1 o 2) para cada dato
clusters = pipeline.named_steps['clustering'].labels_

# Obtenemos el dataframe de características LRMFP (aplicamos paso 1 del pipeline)
df_caracteristicas = (pipeline.named_steps['caracteristicas'].fit_transform(df_retail))

# Obtenemos los datos escalados (aplicamos paso 2 del pipeline)
X_escalamiento = pipeline.named_steps['escalamiento'].fit_transform(df_caracteristicas)
df_escalamiento = pd.DataFrame({
    'Length':X_escalamiento[:,0],
    'Recency':X_escalamiento[:,1],
    'Monetary':X_escalamiento[:,2],
    'Frecuency':X_escalamiento[:,3],
    'Periodicity':X_escalamiento[:,4]
})

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Agregamos los clusters respectivos de cada dato al dataframe escalado:

In [13]:
df_escalamiento_labels = (
    pd.concat([df_escalamiento, pd.DataFrame(clusters, columns=['Cluster'])], axis=1)
)
df_escalamiento_labels

,Length,Recency,Monetary,Frecuency,Periodicity,Cluster
0,0.525469,0.439678,0.004658,0.005391,0.121547,1
1,0.099196,0.005362,0.008331,0.012219,0.022099,0
2,0.000000,0.195710,0.004563,0.003055,0.000000,0
3,0.485255,0.112601,0.012109,0.017790,0.088398,1
4,0.000000,0.026810,0.006175,0.003235,0.000000,0
...,...,...,...,...,...,...
4161,0.737265,0.045576,0.000430,0.038455,0.066298,1
4162,0.000000,0.176944,0.007255,0.004492,0.000000,0
4163,0.000000,0.790885,0.016808,0.001617,0.000000,2
4164,0.662198,0.297587,0.008686,0.011500,0.165746,1


Agrupando según cluster calculamos los promedios para cada uno de los atributos:

In [14]:
promedios_por_cluster = df_escalamiento_labels.groupby('Cluster').mean()
promedios_por_cluster

,Length,Recency,Monetary,Frecuency,Periodicity
Cluster,,,,,
0,0.111857,0.141842,0.011000,0.008620,0.030439
1,0.746528,0.096804,0.014933,0.029631,0.116944
2,0.068748,0.669955,0.011709,0.005046,0.021742


Sabemos por las definiciones vistas al inicio que a mayor valor de Length (L), Monetary (M), Frecuency (F), Periodicity (P) y menor valor de Recency (R) mayor es la fidelidad del cliente. Con esto, de la tabla podemos observar claramente los 3 grupos formados de clientes:

- Cluster 1: Clientes muy fieles. Tienen los valores más grandes de las características LMFP y el menor valor para la característica R.
- Cluster 0: Cliente regulares. Tienen el segundo valor más alto para las características LFP y el segundo menor valor para la característica R.
- Cluster 2: Clientes ocasionales. Tienen el valor más bajo para las características LFP y el valor más alto (por mucho) para la característica R (lo que prueba el desinterés de los usuarios).

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [18]:
# Agregamos los clusters respectivos de cada dato al dataframe t-SNE
df_tsne_labels = (
    pd.concat([df_tsne, pd.DataFrame(clusters, columns=['Cluster'])], axis=1)
)
df_tsne_labels

,Componente 1,Componente 2,Cluster
0,-6.092624,16.849468,1
1,-8.642876,-39.352886,0
2,-58.655277,-26.779863,0
3,10.007330,-16.572449,1
4,15.210716,-51.652348,0
...,...,...,...
4161,36.589722,-14.772819,1
4162,-51.869366,-36.514755,0
4163,-7.680365,46.186344,2
4164,10.793542,15.500110,1


In [22]:
# Visualizamos los puntos y sus clusters
fig = px.scatter(df_tsne_labels, x='Componente 1', y='Componente 2',color='Cluster')
fig.update_layout(
    title = 'Clusters de Clientes'
)
fig.show()

Los clusters no se separan de manera perfecta, especialmente el cluster 1 (en rojo) de clientes muy fieles del cluster 0 (en azul) de clientes regulares. De todas formas se aprecia que el cluster 1 tiene los puntos menos dispersos que el resto de los clusters 2 (en amarillo) y 0, pero la decisión en los bordes no es muy clara, es decir, no queda claro a qué cluster asignar un punto que está alrededor del medio del gráfico (en la frontera del cluster 0 y 1). Respecto al cluster 2 de clientes ocasionales, está bien separado del resto en general, salvo cuando se acerca al medio del gráfico que es donde surge el problema de las fronteras mencionado. Este motivo nos lleva a pensar que quizás el algoritmo de clustering de K-Means no es el más adecuado para este dataset, por lo que queda propuesto para futuro verificar qué sucede con otros algoritmos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>